In [15]:
import sys
sys.path.append('../')
from model.VQAE import VQAE
from utils import device, add_noise
from tqdm import tqdm
import torch
from data.FloorPlanLoader import *
import torch.nn.functional as F
import random
import json
import torchvision
import matplotlib.pyplot as plt
import geopandas as gpd

USE_MULTISCALE = True
USE_MULTITASK = True

#Reproducability Checks:
random.seed(0) #Python
torch.manual_seed(0) #Torch
np.random.seed(0) #NumPy

import re
import torchvision
import torchvision.transforms.functional as TF
import imageio
from torchvision.utils import save_image
# %load_ext autoreload
# %autoreload 2
import csv
import numpy
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import seaborn as sns
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [5]:
#Hyperparameter
device = 'cpu'
batch_size = 128
n_hiddens = 32
n_residual_hiddens = 32
n_residual_layers = 1
embedding_dim = 64
n_embeddings = 218
beta = .25
lr = 3e-3
noise=False
noise_weight=0.05
img_channel=3
vqae = VQAE(n_hiddens, n_residual_hiddens, n_residual_layers,
                n_embeddings, embedding_dim, 
                beta, img_channel).to(device)
checkpoint = torch.load("../best_checkpoint/final/55-vqae-0.04753296934928414.pt", map_location=torch.device('cpu'))
vqae.load_state_dict(checkpoint)
for param in vqae.parameters():
    param.to('cpu')

In [6]:
curr_data = 'data20_right'



In [12]:
folder_path = 'kmeans_5_cluster'
files_to_process = []

for filename in os.listdir(folder_path):
    if filename.endswith('area_aggregation.csv'):
        files_to_process.append(os.path.join(folder_path, filename))
files_to_process

['kmeans_5_cluster\\MARL_data00_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\MARL_data00_right_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\MARL_data10_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\MARL_data10_right_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\MARL_data20_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\MARL_data20_right_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data00_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data00_right_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data10_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data10_right_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data20_left_5cluster_area_aggregation.csv',
 'kmeans_5_cluster\\VQAE_data20_right_5cluster_area_aggregation.csv']

In [33]:
for file_path in files_to_process:
    curr_data = file_path.split("5_cluster_area")[0].split('kmeans_5_cluster\\')[1].split('_5cluster')[0][5:]
    print(curr_data)
    floor = FloorPlanDataset(multi_scale=True, root=f'../data/data_root/{curr_data}/',\
                             data_config='../data/data_config/', preprocess=True)
    
    df = pd.read_csv(file_path, index_col=None)
    df['index_png'] = df['index'].apply(lambda x: calculate_index_png(x))
    df.to_csv(file_path, index=False)

data00_left
data00_right
data10_left
data10_right
data20_left
data20_right
data00_left
data00_right
data10_left
data10_right
data20_left
data20_right
